<a href="https://colab.research.google.com/github/wendy60/Hybrid-recommender-system/blob/second-submit/baseline_model_NAML_Neural_News_Recommendation_with_Attentive_Multi_View_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

I use an open source project from github, so I need to declare the copyright for each model. I use the MIND public dataset and the python package -- recommenders from microsoft.

Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

# **Global Setting and imports**

In [ ]:
pip install recommenders

     |████████████████████████████████| 314 kB 4.9 MB/s 
     |████████████████████████████████| 310 kB 76.4 MB/s 
     |████████████████████████████████| 59 kB 8.9 MB/s 
     |████████████████████████████████| 1.5 MB 78.6 MB/s 
     |████████████████████████████████| 11.8 MB 28.7 MB/s 
     |████████████████████████████████| 22.3 MB 1.3 MB/s 
     |████████████████████████████████| 93 kB 2.7 MB/s 
     |████████████████████████████████| 61 kB 11.5 MB/s 
     |████████████████████████████████| 12.4 MB 72.6 MB/s 
     |████████████████████████████████| 3.3 MB 44.9 MB/s 
     |████████████████████████████████| 636 kB 65.1 MB/s 
     |████████████████████████████████| 749 kB 54.9 MB/s 
     |████████████████████████████████| 895 kB 53.1 MB/s 
     |████████████████████████████████| 3.3 MB 49.7 MB/s 
     |████████████████████████████████| 61 kB 690 kB/s 
  Created wheel for lightfm: filename=lightfm-1.16-cp37-cp37m-linux_x86_64.whl size=706150 sha256=d81c30fd44ef83549a2d861fff177f98cd972b

In [ ]:
pip install tensorflow-gpu==1.15.2

     |████████████████████████████████| 410.9 MB 29 kB/s 
     |████████████████████████████████| 50 kB 9.0 MB/s 
     |████████████████████████████████| 503 kB 37.5 MB/s 
     |████████████████████████████████| 3.8 MB 35.6 MB/s 
  Created wheel for gast: filename=gast-0.2.2-py3-none-any.whl size=7554 sha256=294c415ade0c9553a61ed61f11ce48ef495b14dc1a951685da7526c9cc5fce04
  Stored in directory: /root/.cache/pip/wheels/21/7f/02/420f32a803f7d0967b48dd823da3f558c5166991bfd204eef3
Successfully built gast
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.7.0
    Uninstalling tensorflow-estimator-2.7.0:
      Successfully uninstalled tensorflow-estimator-2.7.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.7.0
    Uninstalling tensorboard-2.7.0:
      Successfully uninstalled tensorboard-2.7.0
  Attempting uninstall: gast
    Found existing installation: gast 0.4.0
    Uninstalling gast-0.4.0:
      Succes

In [ ]:
import sys
import os
import numpy as np
import zipfile
from tqdm import tqdm
from tempfile import TemporaryDirectory
import tensorflow as tf
tf.get_logger().setLevel('ERROR') # only show error messages

from recommenders.models.deeprec.deeprec_utils import download_deeprec_resources 
from recommenders.models.newsrec.newsrec_utils import prepare_hparams
from recommenders.models.newsrec.models.naml import NAMLModel
from recommenders.models.newsrec.io.mind_all_iterator import MINDAllIterator
from recommenders.models.newsrec.newsrec_utils import get_mind_data_set

print("System version: {}".format(sys.version))
print("Tensorflow version: {}".format(tf.__version__))

System version: 3.7.12 (default, Sep 10 2021, 00:21:48) 
[GCC 7.5.0]
Tensorflow version: 1.15.2


# **Prepare Parameters**

In [ ]:
epochs = 5
seed = 42
batch_size = 32

# Options: demo, small, large
MIND_type = 'demo'

# **Download and load data**

In [ ]:
tmpdir = TemporaryDirectory()
data_path = tmpdir.name
##train data
train_news_file = os.path.join(data_path, 'train', r'news.tsv')
train_behaviors_file = os.path.join(data_path, 'train', r'behaviors.tsv')
##validation data
valid_news_file = os.path.join(data_path, 'valid', r'news.tsv')
valid_behaviors_file = os.path.join(data_path, 'valid', r'behaviors.tsv')
##word embedding file
wordEmb_file = os.path.join(data_path, "utils", "embedding_all.npy")
##user dictionary file
userDict_file = os.path.join(data_path, "utils", "uid2index.pkl")
##news dictionary files
wordDict_file = os.path.join(data_path, "utils", "word_dict_all.pkl")
vertDict_file = os.path.join(data_path, "utils", "vert_dict.pkl")
subvertDict_file = os.path.join(data_path, "utils", "subvert_dict.pkl")

## configuration file -- yaml
yaml_file = os.path.join(data_path, "utils", r'naml.yaml')

mind_url, mind_train_dataset, mind_dev_dataset, mind_utils = get_mind_data_set(MIND_type)

if not os.path.exists(train_news_file):
    download_deeprec_resources(mind_url, os.path.join(data_path, 'train'), mind_train_dataset)
    
if not os.path.exists(valid_news_file):
    download_deeprec_resources(mind_url, \
                               os.path.join(data_path, 'valid'), mind_dev_dataset)
if not os.path.exists(yaml_file):
    download_deeprec_resources(r'https://recodatasets.z20.web.core.windows.net/newsrec/', \
                               os.path.join(data_path, 'utils'), mind_utils)

100%|██████████| 17.0k/17.0k [00:00<00:00, 27.1kKB/s]
100%|██████████| 9.84k/9.84k [00:00<00:00, 20.5kKB/s]
100%|██████████| 95.0k/95.0k [00:02<00:00, 35.3kKB/s]


# **Create hyper-parameters**

In [ ]:
hparams = prepare_hparams(yaml_file, 
                          wordEmb_file=wordEmb_file,
                          wordDict_file=wordDict_file, 
                          userDict_file=userDict_file,
                          vertDict_file=vertDict_file, 
                          subvertDict_file=subvertDict_file,
                          batch_size=batch_size,
                          vert_num = 19,
                          epochs=epochs)
print(hparams)

data_format=naml,iterator_type=None,support_quick_scoring=True,wordEmb_file=/tmp/tmpwur_2mby/utils/embedding_all.npy,wordDict_file=/tmp/tmpwur_2mby/utils/word_dict_all.pkl,userDict_file=/tmp/tmpwur_2mby/utils/uid2index.pkl,vertDict_file=/tmp/tmpwur_2mby/utils/vert_dict.pkl,subvertDict_file=/tmp/tmpwur_2mby/utils/subvert_dict.pkl,title_size=30,body_size=50,word_emb_dim=300,word_size=None,user_num=None,vert_num=19,subvert_num=249,his_size=50,npratio=4,dropout=0.2,attention_hidden_dim=200,head_num=4,head_dim=100,cnn_activation=relu,dense_activation=relu,filter_num=400,window_size=3,vert_emb_dim=100,subvert_emb_dim=100,gru_unit=400,type=ini,user_emb_dim=50,learning_rate=0.0001,loss=cross_entropy_loss,optimizer=adam,epochs=5,batch_size=32,show_step=100000,metrics=['group_auc', 'mean_mrr', 'ndcg@5;10']


In [ ]:
iterator = MINDAllIterator

# **Train the NAML model**

In [ ]:
model = NAMLModel(hparams, iterator, seed=seed)


In [ ]:
print(model.run_eval(valid_news_file, valid_behaviors_file))


18693it [01:08, 274.42it/s]
7507it [00:26, 283.06it/s]
7538it [00:01, 5010.61it/s]


{'group_auc': 0.482, 'mean_mrr': 0.2107, 'ndcg@5': 0.2149, 'ndcg@10': 0.2777}


In [ ]:
%%time
model.fit(train_news_file, train_behaviors_file,valid_news_file, valid_behaviors_file)


1085it [02:30,  7.22it/s]
18693it [01:03, 296.21it/s]
7507it [00:26, 288.24it/s]
7538it [00:01, 4914.39it/s]


at epoch 1
train info: logloss loss:1.3007966629920467
eval info: group_auc:0.6449, mean_mrr:0.2948, ndcg@10:0.3926, ndcg@5:0.3273
at epoch 1 , train time: 150.3 eval time: 99.6


1085it [02:30,  7.22it/s]
18693it [01:03, 294.90it/s]
7507it [00:26, 288.58it/s]
7538it [00:01, 5063.21it/s]


at epoch 2
train info: logloss loss:1.2934580073927955
eval info: group_auc:0.6492, mean_mrr:0.3058, ndcg@10:0.4008, ndcg@5:0.3356
at epoch 2 , train time: 150.3 eval time: 99.9


1085it [02:30,  7.21it/s]
18693it [01:02, 300.76it/s]
7507it [00:26, 285.80it/s]
7538it [00:01, 4898.52it/s]


at epoch 3
train info: logloss loss:1.2792713094966204
eval info: group_auc:0.6505, mean_mrr:0.2996, ndcg@10:0.394, ndcg@5:0.3281
at epoch 3 , train time: 150.4 eval time: 98.7


1085it [02:31,  7.15it/s]
18693it [01:03, 293.96it/s]
7507it [00:26, 285.47it/s]
7538it [00:01, 4927.20it/s]


at epoch 4
train info: logloss loss:1.2672772317987433
eval info: group_auc:0.6533, mean_mrr:0.3029, ndcg@10:0.3982, ndcg@5:0.3333
at epoch 4 , train time: 151.7 eval time: 100.2


1085it [02:30,  7.19it/s]
18693it [01:03, 293.81it/s]
7507it [00:26, 288.17it/s]
7538it [00:01, 5014.21it/s]


at epoch 5
train info: logloss loss:1.253925045349631
eval info: group_auc:0.655, mean_mrr:0.3033, ndcg@10:0.3994, ndcg@5:0.3333
at epoch 5 , train time: 151.0 eval time: 100.0
CPU times: user 21min 25s, sys: 1min 37s, total: 23min 3s
Wall time: 20min 52s


In [ ]:
%%time
res_syn = model.run_eval(valid_news_file, valid_behaviors_file)
print(res_syn)

18693it [01:03, 293.55it/s]
7507it [00:26, 287.28it/s]
7538it [00:01, 5115.77it/s]


{'group_auc': 0.655, 'mean_mrr': 0.3033, 'ndcg@5': 0.3333, 'ndcg@10': 0.3994}
CPU times: user 2min 2s, sys: 15.3 s, total: 2min 17s
Wall time: 1min 40s
